## Lib

In [1]:
!git clone https://github.com/DatenBiene/Medical_Time_Series_Classification.git

Cloning into 'Medical_Time_Series_Classification'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 367 (delta 54), reused 77 (delta 23), pack-reused 241
Receiving objects: 100% (367/367), 26.48 MiB | 24.08 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [2]:
!pip install sktime

     |████████████████████████████████| 5.0MB 3.4MB/s 
  Created wheel for sktime: filename=sktime-0.3.0-cp36-cp36m-linux_x86_64.whl size=3299841 sha256=6da6980a82b05488dcb86fffd5f4771302ab9d232449db8013a52876f826c876
  Stored in directory: /root/.cache/pip/wheels/92/5a/c2/859f666f521c12e592cf689e8e2ebff32d74d0416b74dad213
Successfully built sktime


In [0]:
import os 
os.chdir('/content/Medical_Time_Series_Classification')

In [4]:
import pandas as pd
import numpy as np

from sktime.classifiers.distance_based import KNeighborsTimeSeriesClassifier

from utils.evaluation import evaluate, get_binary_metrics
from utils.utils import read_dataset,split_dataset

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


## Transplant

In [14]:
%%time
x_train,y_train,x_val,y_val,y_val_true,x_test,y_test,_, col = np.load('data/transplant_1col.npy', allow_pickle=True)

#x_test= np.concatenate([x_test, x_val])
#y_test = np.concatenate([y_test, y_val])

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)

CPU times: user 989 µs, sys: 2.97 ms, total: 3.96 ms
Wall time: 7.33 ms


In [16]:
%%time
knn = KNeighborsTimeSeriesClassifier(metric='dtw')
knn.fit(x_train, y_train)
ypred = knn.predict(x_test)

CPU times: user 1min 52s, sys: 1.66 s, total: 1min 53s
Wall time: 1min 53s


In [17]:
evaluate(y_test,ypred)

,accuracy,recall,precision,f1-score
0,0.484848,0.477444,0.477778,0.477167


In [18]:
get_binary_metrics(y_test,ypred)

,recall,precision,f1-score
0,0.526316,0.555556,0.540541
1,0.428571,0.4,0.413793


## ECG5000

In [19]:
%%time
x_train,y_train,x_val,y_val,x_test,y_test = np.load('data/ECG5000/ECG5000_full.npy', allow_pickle=True)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)

CPU times: user 3.35 ms, sys: 2.02 ms, total: 5.37 ms
Wall time: 7.53 ms


In [20]:
%%time
knn = KNeighborsTimeSeriesClassifier(metric='dtw')
knn.fit(x_train, y_train)
ypred = knn.predict(x_test)

CPU times: user 5min 44s, sys: 35.3 ms, total: 5min 44s
Wall time: 5min 44s


In [21]:
evaluate(y_test,ypred)

,accuracy,recall,precision,f1-score
0,0.956,0.834486,0.776836,0.794125


In [22]:
get_binary_metrics(y_test,ypred)

,recall,precision,f1-score
1,0.996575,0.996575,0.996575
2,0.954802,0.938889,0.946779
3,0.8,0.666667,0.727273
4,0.421053,0.615385,0.5
5,1,0.666667,0.8


## MIT 

In [0]:
x_train,y_train,x_test,y_test = np.load('data/mitbih.npy',allow_pickle=True)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)

In [14]:
%%time
knn = KNeighborsTimeSeriesClassifier(metric='dtw')
knn.fit(x_train, y_train)
ypred = knn.predict(x_test)

CPU times: user 12min 33s, sys: 25.7 s, total: 12min 59s
Wall time: 13min 1s


In [15]:
evaluate(y_test,ypred)

,accuracy,recall,precision,f1-score
0,0.954,0.816178,0.902316,0.851074


In [16]:
get_binary_metrics(y_test,ypred)

,recall,precision,f1-score
0,0.978261,0.973558,0.975904
1,0.692308,0.818182,0.75
2,0.878788,0.805556,0.84058
3,0.666667,1,0.8
4,0.864865,0.914286,0.888889
